# Model for Boeing
Sentiment is built upon the Reuters titles dataset.
Historical data is taken from yahoo finance

In [35]:
from tqdm import tqdm
import numpy as np
import pickle
import nltk
import string
import os
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import time
import sys
import time
from tqdm._tqdm_notebook import tqdm_notebook
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
from datetime import datetime, timedelta
from io import StringIO
import copy

## Scraping historical data from yahoo finance

In [36]:
class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

#requesting crumb and cookie
    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

#requesting data
    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [37]:
# extracting data about Boeing from 400 days back
df_v = YahooFinanceHistory('BA', days_back=4000).get_quote()

In [38]:
#sorting dates from the latest to earliest
df_v.sort_values(by='Date')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-05-15,43.270000,43.990002,42.709999,43.000000,32.646542,5189300
1,2009-05-18,44.250000,44.529999,43.810001,44.369999,33.686687,7288000
2,2009-05-19,44.360001,45.060001,43.900002,44.619999,33.876472,4792100
3,2009-05-20,45.049999,45.740002,44.540001,44.580002,33.846115,4734200
4,2009-05-21,43.990002,44.000000,42.700001,43.290001,32.866718,5785000
...,...,...,...,...,...,...,...
2750,2020-04-20,147.210007,151.630005,142.500000,143.610001,143.610001,34983800
2751,2020-04-21,139.009995,141.899994,135.449997,136.330002,136.330002,27043800
2752,2020-04-22,140.419998,140.820007,134.520004,134.970001,134.970001,22842600
2753,2020-04-23,136.500000,142.380005,136.050003,137.740005,137.740005,25936400


In [39]:
df_v.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [40]:
df_v

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-05-15,43.270000,43.990002,42.709999,43.000000,32.646542,5189300
1,2009-05-18,44.250000,44.529999,43.810001,44.369999,33.686687,7288000
2,2009-05-19,44.360001,45.060001,43.900002,44.619999,33.876472,4792100
3,2009-05-20,45.049999,45.740002,44.540001,44.580002,33.846115,4734200
4,2009-05-21,43.990002,44.000000,42.700001,43.290001,32.866718,5785000
...,...,...,...,...,...,...,...
2750,2020-04-20,147.210007,151.630005,142.500000,143.610001,143.610001,34983800
2751,2020-04-21,139.009995,141.899994,135.449997,136.330002,136.330002,27043800
2752,2020-04-22,140.419998,140.820007,134.520004,134.970001,134.970001,22842600
2753,2020-04-23,136.500000,142.380005,136.050003,137.740005,137.740005,25936400


## Sentiment for all the articles with "Microsoft" in the body of an article

In [41]:
# uploading file
df2 = pd.read_csv(r'C:\Users\Ania Sikora\Desktop\Python - zaawansowane\dfet\sentymenty\tytuły i daty\df_BA.csv')

In [42]:
df2

,Unnamed: 0,Date,compound_mean
0,0,2011-07-06,-0.493900
1,1,2011-07-07,0.074200
2,2,2011-07-08,0.084700
3,3,2011-07-09,0.153100
4,4,2011-07-10,0.055833
...,...,...,...
1528,1528,2017-02-06,0.055833
1529,1529,2017-02-07,-0.273000
1530,1530,2017-02-10,0.055833
1531,1531,2017-02-13,-0.361200


In [43]:
#deleting column Unnamed
df2 = df2.drop(['Unnamed: 0'], axis=1)

In [44]:
df2

,Date,compound_mean
0,2011-07-06,-0.493900
1,2011-07-07,0.074200
2,2011-07-08,0.084700
3,2011-07-09,0.153100
4,2011-07-10,0.055833
...,...,...
1528,2017-02-06,0.055833
1529,2017-02-07,-0.273000
1530,2017-02-10,0.055833
1531,2017-02-13,-0.361200


In [45]:
df2.dtypes

Date              object
compound_mean    float64
dtype: object

In [46]:
#changing column Date type to datetime type
df2.Date=pd.to_datetime(df2['Date'])

In [47]:
#merging dataframe with historical data with dataframe with sentiments 
df3 = pd.merge(df_v,df2,on='Date')

In [48]:
df3

,Date,Open,High,Low,Close,Adj Close,Volume,compound_mean
0,2011-07-06,74.129997,75.160004,74.010002,74.739998,59.897026,3757800,-0.493900
1,2011-07-07,75.330002,76.199997,74.849998,75.989998,60.898773,4976900,0.074200
2,2011-07-08,75.580002,75.580002,74.570000,75.070000,60.161488,4051200,0.084700
3,2011-07-11,74.169998,74.730003,73.000000,73.349998,58.783066,4379000,-0.157993
4,2011-07-12,73.620003,73.860001,71.790001,71.930000,57.645065,5773000,0.360000
...,...,...,...,...,...,...,...,...
1272,2017-02-06,162.419998,164.080002,162.380005,163.979996,152.032944,3110500,0.055833
1273,2017-02-07,165.000000,167.419998,164.869995,166.500000,154.369339,4243200,-0.273000
1274,2017-02-10,165.250000,166.449997,164.470001,166.229996,155.444748,2689700,0.055833
1275,2017-02-13,166.449997,169.070007,166.350006,168.029999,157.127945,3765000,-0.361200


# Machine learning for prediction of label for the next day 

In [49]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import sklearn

In [50]:
#deepcopying dataframe, so there would be no need to run everything from the beggining
df = copy.deepcopy(df3)

In [51]:
df['Label'] = ''

In [52]:
#label indicates wheter the price will go up(1) or down(0) next day
m = 1
for i in df.Close:
    if i < df.Close[m]:
        df['Label'][m-1] = 1
    else:
        df['Label'][m-1] = 0
    m = m+1

C:\Users\Ania Sikora\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Ania Sikora\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


KeyError: 1277

In [53]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,compound_mean,Label
0,2011-07-06,74.129997,75.160004,74.010002,74.739998,59.897026,3757800,-0.493900,1
1,2011-07-07,75.330002,76.199997,74.849998,75.989998,60.898773,4976900,0.074200,0
2,2011-07-08,75.580002,75.580002,74.570000,75.070000,60.161488,4051200,0.084700,0
3,2011-07-11,74.169998,74.730003,73.000000,73.349998,58.783066,4379000,-0.157993,0
4,2011-07-12,73.620003,73.860001,71.790001,71.930000,57.645065,5773000,0.360000,1
...,...,...,...,...,...,...,...,...,...
1272,2017-02-06,162.419998,164.080002,162.380005,163.979996,152.032944,3110500,0.055833,1
1273,2017-02-07,165.000000,167.419998,164.869995,166.500000,154.369339,4243200,-0.273000,0
1274,2017-02-10,165.250000,166.449997,164.470001,166.229996,155.444748,2689700,0.055833,1
1275,2017-02-13,166.449997,169.070007,166.350006,168.029999,157.127945,3765000,-0.361200,1


In [54]:
#dropping the index
df.drop(index = 1276, inplace = True)

In [55]:
df.dtypes

Date             datetime64[ns]
Open                    float64
High                    float64
Low                     float64
Close                   float64
Adj Close               float64
Volume                    int64
compound_mean           float64
Label                    object
dtype: object

In [56]:
#converting column Label to numeric
df.Label=pd.to_numeric(df['Label'])

In [57]:
df.dtypes

Date             datetime64[ns]
Open                    float64
High                    float64
Low                     float64
Close                   float64
Adj Close               float64
Volume                    int64
compound_mean           float64
Label                     int64
dtype: object

In [58]:
array = df.values

In [59]:
#creating training and testing datasets
X = array[:,1:8]
Y = array[:,8]

In [60]:
#standardising features, fitting and transforming X
X = sklearn.preprocessing.MinMaxScaler().fit_transform(X)

In [61]:
#casting Y to data type integer
Y = Y.astype('int')

In [62]:
print(X[0])
print(df.columns[1:8])

[0.15129418 0.14381154 0.16181233 0.15513382 0.12252169 0.06436869
 0.1636901 ]
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'compound_mean'], dtype='object')


In [63]:
#choosing best features for the model
test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(X, Y)
np.set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
print(features[0:5,:])

[0.05  0.05  0.047 0.046 0.017 0.085 0.235]
[[0.151 0.144 0.162 0.064 0.164]
 [0.162 0.153 0.169 0.092 0.521]
 [0.165 0.148 0.167 0.071 0.528]
 [0.152 0.14  0.153 0.079 0.375]
 [0.147 0.132 0.142 0.111 0.701]]


In [64]:
#Open, High, Low, Volume and compound mean give the most information
features

array([[0.151, 0.144, 0.162, 0.064, 0.164],
       [0.162, 0.153, 0.169, 0.092, 0.521],
       [0.165, 0.148, 0.167, 0.071, 0.528],
       ...,
       [0.979, 0.977, 0.979, 0.076, 0.303],
       [0.982, 0.968, 0.975, 0.04 , 0.509],
       [0.993, 0.992, 0.992, 0.065, 0.247]])

In [65]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

#building and training a model
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.25)
model = Sequential()
model.add(Dense(16, input_shape=(5,) ))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data= (X_test,y_test),
                              callbacks=[es])

Train on 957 samples, validate on 319 samples
Epoch 1/100
957/957 [==============================] - ETA: 1:03 - loss: 0.6897 - accuracy: 0.46 - ETA: 32s - loss: 0.6769 - accuracy: 0.6094 - ETA: 15s - loss: 0.6847 - accuracy: 0.578 - ETA: 4s - loss: 0.6923 - accuracy: 0.521 - ETA: 2s - loss: 0.6928 - accuracy: 0.52 - ETA: 0s - loss: 0.6949 - accuracy: 0.52 - ETA: 0s - loss: 0.6990 - accuracy: 0.53 - 3s 3ms/step - loss: 0.6977 - accuracy: 0.5298 - val_loss: 0.6936 - val_accuracy: 0.5047
Epoch 2/100
957/957 [==============================] - ETA: 0s - loss: 0.7032 - accuracy: 0.37 - ETA: 0s - loss: 0.7043 - accuracy: 0.44 - ETA: 0s - loss: 0.6987 - accuracy: 0.46 - ETA: 0s - loss: 0.6959 - accuracy: 0.47 - ETA: 0s - loss: 0.6953 - accuracy: 0.48 - ETA: 0s - loss: 0.6958 - accuracy: 0.49 - 0s 353us/step - loss: 0.6957 - accuracy: 0.4932 - val_loss: 0.6956 - val_accuracy: 0.5047
Epoch 3/100
957/957 [==============================] - ETA: 0s - loss: 0.7022 - accuracy: 0.37 - ETA: 1s - loss:

Epoch 20/100
957/957 [==============================] - ETA: 0s - loss: 0.7128 - accuracy: 0.34 - ETA: 0s - loss: 0.6894 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6926 - accuracy: 0.51 - ETA: 0s - loss: 0.6921 - accuracy: 0.52 - ETA: 0s - loss: 0.6905 - accuracy: 0.53 - 0s 406us/step - loss: 0.6900 - accuracy: 0.5392 - val_loss: 0.6964 - val_accuracy: 0.5047
Epoch 21/100
957/957 [==============================] - ETA: 0s - loss: 0.7043 - accuracy: 0.46 - ETA: 0s - loss: 0.6874 - accuracy: 0.56 - ETA: 0s - loss: 0.6955 - accuracy: 0.52 - ETA: 0s - loss: 0.6966 - accuracy: 0.51 - ETA: 0s - loss: 0.6951 - accuracy: 0.51 - ETA: 0s - loss: 0.6944 - accuracy: 0.51 - ETA: 0s - loss: 0.6934 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - 1s 686us/step - loss: 0.6901 - accuracy: 0.5444 - val_loss: 0.6968 - val_accuracy: 0.5047
Epoch 22/100
957/957 [==============

957/957 [==============================] - ETA: 0s - loss: 0.7210 - accuracy: 0.37 - ETA: 0s - loss: 0.6992 - accuracy: 0.50 - ETA: 0s - loss: 0.6911 - accuracy: 0.51 - ETA: 0s - loss: 0.6907 - accuracy: 0.51 - ETA: 0s - loss: 0.6902 - accuracy: 0.52 - ETA: 0s - loss: 0.6904 - accuracy: 0.52 - ETA: 0s - loss: 0.6908 - accuracy: 0.52 - ETA: 0s - loss: 0.6907 - accuracy: 0.52 - ETA: 0s - loss: 0.6894 - accuracy: 0.53 - ETA: 0s - loss: 0.6896 - accuracy: 0.53 - 1s 725us/step - loss: 0.6896 - accuracy: 0.5381 - val_loss: 0.6969 - val_accuracy: 0.4984
Epoch 36/100
957/957 [==============================] - ETA: 0s - loss: 0.6806 - accuracy: 0.59 - ETA: 0s - loss: 0.6809 - accuracy: 0.57 - ETA: 0s - loss: 0.6820 - accuracy: 0.56 - ETA: 0s - loss: 0.6883 - accuracy: 0.53 - ETA: 0s - loss: 0.6857 - accuracy: 0.54 - ETA: 0s - loss: 0.6866 - accuracy: 0.54 - ETA: 0s - loss: 0.6843 - accuracy: 0.55 - ETA: 0s - loss: 0.6865 - accuracy: 0.54 - ETA: 0s - loss: 0.6878 - accuracy: 0.53 - 1s 568us/step

957/957 [==============================] - ETA: 0s - loss: 0.6888 - accuracy: 0.50 - ETA: 0s - loss: 0.6923 - accuracy: 0.51 - ETA: 0s - loss: 0.6945 - accuracy: 0.50 - ETA: 0s - loss: 0.6919 - accuracy: 0.51 - ETA: 0s - loss: 0.6899 - accuracy: 0.52 - ETA: 0s - loss: 0.6896 - accuracy: 0.53 - 0s 379us/step - loss: 0.6879 - accuracy: 0.5381 - val_loss: 0.6985 - val_accuracy: 0.4953
Epoch 53/100
957/957 [==============================] - ETA: 0s - loss: 0.7026 - accuracy: 0.46 - ETA: 0s - loss: 0.6987 - accuracy: 0.48 - ETA: 0s - loss: 0.6945 - accuracy: 0.52 - ETA: 0s - loss: 0.6953 - accuracy: 0.50 - ETA: 0s - loss: 0.6924 - accuracy: 0.52 - ETA: 0s - loss: 0.6889 - accuracy: 0.54 - ETA: 0s - loss: 0.6886 - accuracy: 0.54 - 0s 420us/step - loss: 0.6890 - accuracy: 0.5434 - val_loss: 0.6999 - val_accuracy: 0.4890
Epoch 54/100
957/957 [==============================] - ETA: 0s - loss: 0.7215 - accuracy: 0.46 - ETA: 0s - loss: 0.6962 - accuracy: 0.50 - ETA: 0s - loss: 0.6842 - accuracy: 

957/957 [==============================] - ETA: 0s - loss: 0.6992 - accuracy: 0.43 - ETA: 0s - loss: 0.6753 - accuracy: 0.58 - ETA: 0s - loss: 0.6756 - accuracy: 0.57 - ETA: 0s - loss: 0.6820 - accuracy: 0.55 - ETA: 0s - loss: 0.6824 - accuracy: 0.56 - ETA: 0s - loss: 0.6817 - accuracy: 0.55 - ETA: 0s - loss: 0.6873 - accuracy: 0.54 - ETA: 0s - loss: 0.6879 - accuracy: 0.54 - ETA: 0s - loss: 0.6884 - accuracy: 0.54 - 1s 651us/step - loss: 0.6892 - accuracy: 0.5392 - val_loss: 0.6982 - val_accuracy: 0.4984
Epoch 71/100
957/957 [==============================] - ETA: 0s - loss: 0.7014 - accuracy: 0.56 - ETA: 0s - loss: 0.6919 - accuracy: 0.54 - ETA: 0s - loss: 0.6925 - accuracy: 0.54 - ETA: 0s - loss: 0.6928 - accuracy: 0.54 - ETA: 0s - loss: 0.6891 - accuracy: 0.55 - ETA: 0s - loss: 0.6884 - accuracy: 0.56 - ETA: 0s - loss: 0.6873 - accuracy: 0.56 - ETA: 0s - loss: 0.6891 - accuracy: 0.55 - ETA: 0s - loss: 0.6890 - accuracy: 0.54 - 1s 562us/step - loss: 0.6884 - accuracy: 0.5496 - val_l

957/957 [==============================] - ETA: 0s - loss: 0.7157 - accuracy: 0.46 - ETA: 0s - loss: 0.6784 - accuracy: 0.55 - ETA: 0s - loss: 0.6824 - accuracy: 0.53 - ETA: 0s - loss: 0.6812 - accuracy: 0.53 - ETA: 0s - loss: 0.6856 - accuracy: 0.51 - ETA: 0s - loss: 0.6894 - accuracy: 0.50 - ETA: 0s - loss: 0.6893 - accuracy: 0.51 - ETA: 0s - loss: 0.6909 - accuracy: 0.52 - ETA: 0s - loss: 0.6904 - accuracy: 0.52 - ETA: 0s - loss: 0.6907 - accuracy: 0.52 - ETA: 0s - loss: 0.6898 - accuracy: 0.53 - ETA: 0s - loss: 0.6889 - accuracy: 0.54 - 1s 892us/step - loss: 0.6884 - accuracy: 0.5496 - val_loss: 0.6967 - val_accuracy: 0.4953
Epoch 85/100
957/957 [==============================] - ETA: 0s - loss: 0.6435 - accuracy: 0.71 - ETA: 0s - loss: 0.6819 - accuracy: 0.57 - ETA: 0s - loss: 0.6853 - accuracy: 0.54 - ETA: 0s - loss: 0.6896 - accuracy: 0.54 - ETA: 0s - loss: 0.6829 - accuracy: 0.56 - ETA: 0s - loss: 0.6855 - accuracy: 0.54 - ETA: 0s - loss: 0.6843 - accuracy: 0.55 - ETA: 0s - los

Epoch 98/100
957/957 [==============================] - ETA: 0s - loss: 0.6965 - accuracy: 0.53 - ETA: 0s - loss: 0.6845 - accuracy: 0.56 - ETA: 0s - loss: 0.6856 - accuracy: 0.55 - ETA: 0s - loss: 0.6858 - accuracy: 0.55 - ETA: 0s - loss: 0.6887 - accuracy: 0.54 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - ETA: 0s - loss: 0.6881 - accuracy: 0.53 - ETA: 0s - loss: 0.6875 - accuracy: 0.53 - 0s 490us/step - loss: 0.6872 - accuracy: 0.5381 - val_loss: 0.6966 - val_accuracy: 0.4922
Epoch 99/100
957/957 [==============================] - ETA: 0s - loss: 0.7174 - accuracy: 0.40 - ETA: 0s - loss: 0.6822 - accuracy: 0.55 - ETA: 0s - loss: 0.6840 - accuracy: 0.54 - ETA: 0s - loss: 0.6819 - accuracy: 0.55 - ETA: 0s - loss: 0.6834 - accuracy: 0.55 - ETA: 0s - loss: 0.6834 - accuracy: 0.56 - ETA: 0s - loss: 0.6826 - accuracy: 0.55 - ETA: 0s - loss: 0.6862 - accuracy: 0.54 - ETA: 0s - loss: 0.6863 - accuracy: 0.54 - 1s 684us/step - loss: 0.6885 - accuracy: 0.5371 - val_loss: 0.6975 - val_accuracy: 0

In [66]:
#evaluating the model
model.evaluate(X_test, y_test)

319/319 [==============================] - ETA:  - 0s 129us/step


[0.6955296273889213, 0.4952978193759918]

# LSTM AND GRU METHOD

In [76]:
normalized_df1 = copy.deepcopy(df3)

In [77]:
normalized_df1 = normalized_df1[['Open','High','Low','Volume', 'Close','compound_mean']]

In [78]:
#normalizing function
def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

In [79]:
#for later
normalized_df2 = copy.deepcopy(normalized_df1)
normalized_df3 = copy.deepcopy(normalized_df1)

In [80]:
normalized_df2

,Open,High,Low,Volume,Close,compound_mean
0,74.129997,75.160004,74.010002,3757800,74.739998,-0.493900
1,75.330002,76.199997,74.849998,4976900,75.989998,0.074200
2,75.580002,75.580002,74.570000,4051200,75.070000,0.084700
3,74.169998,74.730003,73.000000,4379000,73.349998,-0.157993
4,73.620003,73.860001,71.790001,5773000,71.930000,0.360000
...,...,...,...,...,...,...
1272,162.419998,164.080002,162.380005,3110500,163.979996,0.055833
1273,165.000000,167.419998,164.869995,4243200,166.500000,-0.273000
1274,165.250000,166.449997,164.470001,2689700,166.229996,0.055833
1275,166.449997,169.070007,166.350006,3765000,168.029999,-0.361200


In [81]:
mean = normalized_df1.mean(axis = 0)
normalized_df1 -= mean
std = normalized_df1.std(axis=0)
normalized_df1 /= std

In [82]:
#adding label: up or down or steady
def add_label(df):
    idx = len(df.columns)
    new_col = np.where(df['Close'] >= df['Close'].shift(1), 1, 0)  
    df.insert(loc=idx, column='Label', value=new_col)
    df = df.fillna(0)

In [83]:
add_label(normalized_df1)

In [84]:
normalized_df1 = normalized_df1.values

In [85]:
normalized_df1

array([[-1.289, -1.278, -1.268, ..., -1.269, -1.929,  0.   ],
       [-1.249, -1.243, -1.24 , ..., -1.228,  0.029,  1.   ],
       [-1.241, -1.264, -1.25 , ..., -1.258,  0.065,  0.   ],
       ...,
       [ 1.743,  1.739,  1.758, ...,  1.771, -0.035,  0.   ],
       [ 1.783,  1.825,  1.821, ...,  1.831, -1.472,  1.   ],
       [ 1.824,  1.816,  1.85 , ...,  1.846, -0.035,  1.   ]])

In [86]:
from keras.utils import to_categorical

#defining our generator
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=32, step=5):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][-1]
        yield samples, to_categorical(targets)

In [87]:
lookback = 30
step = 10
delay = 1
batch_size = 32

In [88]:
#Splitting data into train, test and validation set
train_gen = generator(normalized_df1,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=round(0.6*len(normalized_df1)),
                      shuffle=False,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(normalized_df1,
                    lookback=lookback,
                    delay=delay,
                    min_index=round(0.6*len(normalized_df1))+1,
                    max_index=round(0.8*len(normalized_df1)),
                    step=step,
                    batch_size=batch_size)
test_gen = generator(normalized_df1,
                     lookback=lookback,
                     delay=delay,
                     min_index=round(0.8*len(normalized_df1))+1,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (round(0.8*len(normalized_df1)) - round(0.6*len(normalized_df1))+1 - lookback) # how many steps to draw from val_gen in order to see the entire validation set
test_steps = (len(normalized_df1) - round(0.8*len(normalized_df1))+1 - lookback)
# How many steps to draw from test_gen in order to see the entire test set

In [89]:
#Building and training LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True,
                    input_shape=(None, normalized_df1.shape[-1]),
                    kernel_initializer='random_uniform'))
model.add(Dropout(0.4))
model.add(LSTM(60, dropout=0.0, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(20,activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=250,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/250
2/2 [==============================] - ETA: 3s - loss: 0.2503 - accuracy: 0.50 - 6s 3s/step - loss: 0.2490 - accuracy: 0.5625 - val_loss: 0.2507 - val_accuracy: 0.4911
Epoch 2/250
2/2 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.56 - 1s 674ms/step - loss: 0.2482 - accuracy: 0.5469 - val_loss: 0.2482 - val_accuracy: 0.4915
Epoch 3/250
2/2 [==============================] - ETA: 0s - loss: 0.2568 - accuracy: 0.40 - 1s 692ms/step - loss: 0.2538 - accuracy: 0.4062 - val_loss: 0.2502 - val_accuracy: 0.4281
Epoch 4/250
2/2 [==============================] - ETA: 0s - loss: 0.2503 - accuracy: 0.53 - 1s 682ms/step - loss: 0.2506 - accuracy: 0.5156 - val_loss: 0.2503 - val_accuracy: 0.4416
Epoch 5/250
2/2 [==============================] - ETA: 0s - loss: 0.2480 - accuracy: 0.59 - 1s 730ms/step - loss: 0.2483 - accuracy: 0.5469 - val_loss: 0.2502 - val_accuracy: 0.4648
Epoch 6/250
2/2 [==============================] - ETA: 0s - loss: 0.2433 - accuracy: 0.

Epoch 46/250
2/2 [==============================] - ETA: 0s - loss: 0.2441 - accuracy: 0.56 - 1s 749ms/step - loss: 0.2429 - accuracy: 0.5625 - val_loss: 0.2546 - val_accuracy: 0.4904
Epoch 47/250
2/2 [==============================] - ETA: 0s - loss: 0.2620 - accuracy: 0.37 - 1s 721ms/step - loss: 0.2546 - accuracy: 0.4219 - val_loss: 0.2522 - val_accuracy: 0.4872
Epoch 48/250
2/2 [==============================] - ETA: 0s - loss: 0.2569 - accuracy: 0.46 - 1s 736ms/step - loss: 0.2505 - accuracy: 0.5469 - val_loss: 0.2531 - val_accuracy: 0.4820
Epoch 49/250
2/2 [==============================] - ETA: 0s - loss: 0.2405 - accuracy: 0.62 - 1s 727ms/step - loss: 0.2421 - accuracy: 0.5781 - val_loss: 0.2570 - val_accuracy: 0.4824
Epoch 50/250
2/2 [==============================] - ETA: 0s - loss: 0.2325 - accuracy: 0.65 - 2s 806ms/step - loss: 0.2338 - accuracy: 0.6406 - val_loss: 0.2532 - val_accuracy: 0.4865
Epoch 51/250
2/2 [==============================] - ETA: 0s - loss: 0.2498 - acc

Epoch 91/250
2/2 [==============================] - ETA: 0s - loss: 0.2559 - accuracy: 0.53 - 2s 753ms/step - loss: 0.2449 - accuracy: 0.6094 - val_loss: 0.2585 - val_accuracy: 0.4375
Epoch 92/250
2/2 [==============================] - ETA: 0s - loss: 0.2673 - accuracy: 0.40 - 1s 708ms/step - loss: 0.2552 - accuracy: 0.4844 - val_loss: 0.2537 - val_accuracy: 0.4375
Epoch 93/250
2/2 [==============================] - ETA: 0s - loss: 0.2431 - accuracy: 0.65 - 1s 680ms/step - loss: 0.2408 - accuracy: 0.6406 - val_loss: 0.2442 - val_accuracy: 0.4331
Epoch 94/250
2/2 [==============================] - ETA: 0s - loss: 0.2371 - accuracy: 0.50 - 1s 678ms/step - loss: 0.2359 - accuracy: 0.5625 - val_loss: 0.2488 - val_accuracy: 0.4330
Epoch 95/250
2/2 [==============================] - ETA: 0s - loss: 0.2486 - accuracy: 0.53 - 1s 730ms/step - loss: 0.2431 - accuracy: 0.5625 - val_loss: 0.2565 - val_accuracy: 0.4369
Epoch 96/250
2/2 [==============================] - ETA: 0s - loss: 0.2351 - acc

2/2 [==============================] - ETA: 0s - loss: 0.2529 - accuracy: 0.56 - 2s 759ms/step - loss: 0.2443 - accuracy: 0.5938 - val_loss: 0.2424 - val_accuracy: 0.4689
Epoch 136/250
2/2 [==============================] - ETA: 0s - loss: 0.2703 - accuracy: 0.46 - 1s 664ms/step - loss: 0.2489 - accuracy: 0.5312 - val_loss: 0.2495 - val_accuracy: 0.4733
Epoch 137/250
2/2 [==============================] - ETA: 0s - loss: 0.2306 - accuracy: 0.75 - 1s 677ms/step - loss: 0.2296 - accuracy: 0.6875 - val_loss: 0.2574 - val_accuracy: 0.4681
Epoch 138/250
2/2 [==============================] - ETA: 0s - loss: 0.2258 - accuracy: 0.56 - 1s 668ms/step - loss: 0.2308 - accuracy: 0.5938 - val_loss: 0.2548 - val_accuracy: 0.4738
Epoch 139/250
2/2 [==============================] - ETA: 0s - loss: 0.2516 - accuracy: 0.46 - 1s 666ms/step - loss: 0.2452 - accuracy: 0.5469 - val_loss: 0.2527 - val_accuracy: 0.4776
Epoch 140/250
2/2 [==============================] - ETA: 0s - loss: 0.2352 - accuracy: 0

2/2 [==============================] - ETA: 0s - loss: 0.2556 - accuracy: 0.50 - 1s 663ms/step - loss: 0.2487 - accuracy: 0.5625 - val_loss: 0.2599 - val_accuracy: 0.4547
Epoch 180/250
2/2 [==============================] - ETA: 0s - loss: 0.2634 - accuracy: 0.53 - 1s 666ms/step - loss: 0.2462 - accuracy: 0.5781 - val_loss: 0.2561 - val_accuracy: 0.4561
Epoch 181/250
2/2 [==============================] - ETA: 0s - loss: 0.2442 - accuracy: 0.62 - 1s 674ms/step - loss: 0.2344 - accuracy: 0.6250 - val_loss: 0.2538 - val_accuracy: 0.4551
Epoch 182/250
2/2 [==============================] - ETA: 0s - loss: 0.2311 - accuracy: 0.56 - 1s 730ms/step - loss: 0.2396 - accuracy: 0.5781 - val_loss: 0.2594 - val_accuracy: 0.4599
Epoch 183/250
2/2 [==============================] - ETA: 0s - loss: 0.2526 - accuracy: 0.46 - 1s 667ms/step - loss: 0.2433 - accuracy: 0.5469 - val_loss: 0.2563 - val_accuracy: 0.4642
Epoch 184/250
2/2 [==============================] - ETA: 0s - loss: 0.2365 - accuracy: 0

2/2 [==============================] - ETA: 0s - loss: 0.2584 - accuracy: 0.46 - 1s 664ms/step - loss: 0.2495 - accuracy: 0.5156 - val_loss: 0.2542 - val_accuracy: 0.4730
Epoch 224/250
2/2 [==============================] - ETA: 0s - loss: 0.2715 - accuracy: 0.53 - 1s 713ms/step - loss: 0.2511 - accuracy: 0.6094 - val_loss: 0.2608 - val_accuracy: 0.4688
Epoch 225/250
2/2 [==============================] - ETA: 0s - loss: 0.2418 - accuracy: 0.68 - 1s 706ms/step - loss: 0.2311 - accuracy: 0.6562 - val_loss: 0.2591 - val_accuracy: 0.4688
Epoch 226/250
2/2 [==============================] - ETA: 0s - loss: 0.2174 - accuracy: 0.56 - 1s 737ms/step - loss: 0.2307 - accuracy: 0.6094 - val_loss: 0.2376 - val_accuracy: 0.4822
Epoch 227/250
2/2 [==============================] - ETA: 0s - loss: 0.2479 - accuracy: 0.50 - 1s 662ms/step - loss: 0.2444 - accuracy: 0.5781 - val_loss: 0.2497 - val_accuracy: 0.4866
Epoch 228/250
2/2 [==============================] - ETA: 0s - loss: 0.2295 - accuracy: 0

In [90]:
test_loss, test_acc = model.evaluate_generator(test_gen, steps=3)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.6145833134651184
test_loss: 0.243784099817276


In [91]:
#Building and training a model with GRU
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.3,
                     recurrent_dropout=0.2,
                     return_sequences=True,
                     input_shape=(None, normalized_df1.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.3,
                     recurrent_dropout=0.1))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)
    
history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=250,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/250
2/2 [==============================] - ETA: 4s - loss: 0.2543 - accuracy: 0.50 - 6s 3s/step - loss: 0.2470 - accuracy: 0.6094 - val_loss: 0.2491 - val_accuracy: 0.4917
Epoch 2/250
2/2 [==============================] - ETA: 0s - loss: 0.2501 - accuracy: 0.46 - 1s 633ms/step - loss: 0.2481 - accuracy: 0.5000 - val_loss: 0.2541 - val_accuracy: 0.4908
Epoch 3/250
2/2 [==============================] - ETA: 0s - loss: 0.2533 - accuracy: 0.53 - 1s 664ms/step - loss: 0.2459 - accuracy: 0.5625 - val_loss: 0.2546 - val_accuracy: 0.4913
Epoch 4/250
2/2 [==============================] - ETA: 0s - loss: 0.2405 - accuracy: 0.59 - 1s 689ms/step - loss: 0.2443 - accuracy: 0.5781 - val_loss: 0.2556 - val_accuracy: 0.4820
Epoch 5/250
2/2 [==============================] - ETA: 0s - loss: 0.2446 - accuracy: 0.53 - 1s 603ms/step - loss: 0.2426 - accuracy: 0.5938 - val_loss: 0.2412 - val_accuracy: 0.4915
Epoch 6/250
2/2 [==============================] - ETA: 0s - loss: 0.2479 - accuracy: 0.

Epoch 46/250
2/2 [==============================] - ETA: 0s - loss: 0.2542 - accuracy: 0.46 - 1s 620ms/step - loss: 0.2449 - accuracy: 0.5312 - val_loss: 0.2525 - val_accuracy: 0.4375
Epoch 47/250
2/2 [==============================] - ETA: 0s - loss: 0.2496 - accuracy: 0.50 - 1s 619ms/step - loss: 0.2399 - accuracy: 0.5781 - val_loss: 0.2468 - val_accuracy: 0.4420
Epoch 48/250
2/2 [==============================] - ETA: 0s - loss: 0.2457 - accuracy: 0.53 - 1s 613ms/step - loss: 0.2461 - accuracy: 0.5000 - val_loss: 0.2482 - val_accuracy: 0.4511
Epoch 49/250
2/2 [==============================] - ETA: 0s - loss: 0.2408 - accuracy: 0.56 - 1s 614ms/step - loss: 0.2456 - accuracy: 0.5938 - val_loss: 0.2557 - val_accuracy: 0.4590
Epoch 50/250
2/2 [==============================] - ETA: 0s - loss: 0.2429 - accuracy: 0.62 - 1s 627ms/step - loss: 0.2467 - accuracy: 0.5625 - val_loss: 0.2518 - val_accuracy: 0.4873
Epoch 51/250
2/2 [==============================] - ETA: 0s - loss: 0.2607 - acc

In [92]:
#Evaluating the model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=3)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5416666865348816
test_loss: 0.2535386383533478


## Without sentiment

In [93]:
#normalizing data
normalized_df1 = normalized_df2

mean = normalized_df1.mean(axis = 0)
normalized_df1 -= mean
std = normalized_df1.std(axis=0)
normalized_df1 /= std
#adding label: up/down or steady
def add_label(df):
    idx = len(df.columns)
    new_col = np.where(df['Close'] >= df['Close'].shift(1), 1, 0)  
    df.insert(loc=idx, column='Label', value=new_col)
    df = df.fillna(0)
    
add_label(normalized_df1)

In [94]:
#applying function 
del normalized_df1['compound_mean']

In [95]:
normalized_df1

,Open,High,Low,Volume,Close,Label
0,-1.288893,-1.277830,-1.268443,-0.336991,-1.269058,0
1,-1.248970,-1.243467,-1.240343,0.130291,-1.227522,1
2,-1.240652,-1.263952,-1.249709,-0.224531,-1.258092,0
3,-1.287562,-1.292038,-1.302230,-0.098885,-1.315245,0
4,-1.305861,-1.320784,-1.342708,0.435436,-1.362429,0
...,...,...,...,...,...,...
1272,1.648477,1.660242,1.687779,-0.585101,1.696228,1
1273,1.734312,1.770602,1.771076,-0.150937,1.779963,1
1274,1.742630,1.738551,1.757695,-0.746394,1.770992,0
1275,1.782553,1.825121,1.820587,-0.334231,1.830803,1


In [96]:
normalized_df1 = normalized_df1.values

In [97]:
#Splitting into tain validation and test set
train_gen = generator(normalized_df1,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=round(0.6*len(normalized_df1)),
                      shuffle=False,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(normalized_df1,
                    lookback=lookback,
                    delay=delay,
                    min_index=round(0.6*len(normalized_df1))+1,
                    max_index=round(0.8*len(normalized_df1)),
                    step=step,
                    batch_size=batch_size)
test_gen = generator(normalized_df1,
                     lookback=lookback,
                     delay=delay,
                     min_index=round(0.8*len(normalized_df1))+1,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (round(0.8*len(normalized_df1)) - round(0.6*len(normalized_df1))+1 - lookback) # how many steps to draw from val_gen in order to see the entire validation set
test_steps = (len(normalized_df1) - round(0.8*len(normalized_df1))+1 - lookback)
# How many steps to draw from test_gen in order to see the entire test set

In [98]:
#Bilding and training LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True,
                    input_shape=(None, normalized_df1.shape[-1]),
                    kernel_initializer='random_uniform'))
model.add(Dropout(0.4))
model.add(LSTM(60, dropout=0.0, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(20,activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=200,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/200
2/2 [==============================] - ETA: 3s - loss: 0.2506 - accuracy: 0.53 - 6s 3s/step - loss: 0.2504 - accuracy: 0.5469 - val_loss: 0.2512 - val_accuracy: 0.4911
Epoch 2/200
2/2 [==============================] - ETA: 0s - loss: 0.2479 - accuracy: 0.50 - 1s 720ms/step - loss: 0.2485 - accuracy: 0.5156 - val_loss: 0.2464 - val_accuracy: 0.4915
Epoch 3/200
2/2 [==============================] - ETA: 0s - loss: 0.2547 - accuracy: 0.40 - 1s 677ms/step - loss: 0.2528 - accuracy: 0.3906 - val_loss: 0.2499 - val_accuracy: 0.4909
Epoch 4/200
2/2 [==============================] - ETA: 0s - loss: 0.2512 - accuracy: 0.34 - 1s 670ms/step - loss: 0.2503 - accuracy: 0.4219 - val_loss: 0.2514 - val_accuracy: 0.4904
Epoch 5/200
2/2 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.59 - 1s 664ms/step - loss: 0.2491 - accuracy: 0.5469 - val_loss: 0.2496 - val_accuracy: 0.4917
Epoch 6/200
2/2 [==============================] - ETA: 0s - loss: 0.2466 - accuracy: 0.

Epoch 46/200
2/2 [==============================] - ETA: 0s - loss: 0.2456 - accuracy: 0.56 - 1s 668ms/step - loss: 0.2474 - accuracy: 0.5469 - val_loss: 0.2541 - val_accuracy: 0.4904
Epoch 47/200
2/2 [==============================] - ETA: 0s - loss: 0.2644 - accuracy: 0.40 - 1s 747ms/step - loss: 0.2642 - accuracy: 0.3594 - val_loss: 0.2491 - val_accuracy: 0.4917
Epoch 48/200
2/2 [==============================] - ETA: 0s - loss: 0.2469 - accuracy: 0.53 - 1s 668ms/step - loss: 0.2505 - accuracy: 0.5156 - val_loss: 0.2512 - val_accuracy: 0.4908
Epoch 49/200
2/2 [==============================] - ETA: 0s - loss: 0.2488 - accuracy: 0.53 - 1s 672ms/step - loss: 0.2470 - accuracy: 0.5312 - val_loss: 0.2539 - val_accuracy: 0.4912
Epoch 50/200
2/2 [==============================] - ETA: 0s - loss: 0.2458 - accuracy: 0.62 - 1s 686ms/step - loss: 0.2419 - accuracy: 0.6250 - val_loss: 0.2526 - val_accuracy: 0.4911
Epoch 51/200
2/2 [==============================] - ETA: 0s - loss: 0.2478 - acc

Epoch 91/200
2/2 [==============================] - ETA: 0s - loss: 0.2618 - accuracy: 0.40 - 1s 658ms/step - loss: 0.2609 - accuracy: 0.4062 - val_loss: 0.2541 - val_accuracy: 0.4912
Epoch 92/200
2/2 [==============================] - ETA: 0s - loss: 0.2498 - accuracy: 0.53 - 1s 665ms/step - loss: 0.2487 - accuracy: 0.5469 - val_loss: 0.2521 - val_accuracy: 0.4911
Epoch 93/200
2/2 [==============================] - ETA: 0s - loss: 0.2500 - accuracy: 0.56 - 1s 665ms/step - loss: 0.2488 - accuracy: 0.5312 - val_loss: 0.2425 - val_accuracy: 0.4915
Epoch 94/200
2/2 [==============================] - ETA: 0s - loss: 0.2406 - accuracy: 0.65 - 1s 669ms/step - loss: 0.2431 - accuracy: 0.6406 - val_loss: 0.2493 - val_accuracy: 0.4909
Epoch 95/200
2/2 [==============================] - ETA: 0s - loss: 0.2498 - accuracy: 0.53 - 1s 732ms/step - loss: 0.2499 - accuracy: 0.5312 - val_loss: 0.2544 - val_accuracy: 0.4904
Epoch 96/200
2/2 [==============================] - ETA: 0s - loss: 0.2381 - acc

2/2 [==============================] - ETA: 0s - loss: 0.2592 - accuracy: 0.40 - 1s 656ms/step - loss: 0.2581 - accuracy: 0.4062 - val_loss: 0.2421 - val_accuracy: 0.4915
Epoch 136/200
2/2 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.53 - 1s 655ms/step - loss: 0.2483 - accuracy: 0.5469 - val_loss: 0.2493 - val_accuracy: 0.4909
Epoch 137/200
2/2 [==============================] - ETA: 0s - loss: 0.2510 - accuracy: 0.56 - 1s 680ms/step - loss: 0.2503 - accuracy: 0.5312 - val_loss: 0.2545 - val_accuracy: 0.4904
Epoch 138/200
2/2 [==============================] - ETA: 0s - loss: 0.2291 - accuracy: 0.65 - 1s 740ms/step - loss: 0.2381 - accuracy: 0.6406 - val_loss: 0.2494 - val_accuracy: 0.4917
Epoch 139/200
2/2 [==============================] - ETA: 0s - loss: 0.2493 - accuracy: 0.53 - 1s 662ms/step - loss: 0.2480 - accuracy: 0.5312 - val_loss: 0.2508 - val_accuracy: 0.4908
Epoch 140/200
2/2 [==============================] - ETA: 0s - loss: 0.2365 - accuracy: 0

Epoch 00178: early stopping


In [99]:
#evaluating the model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=4)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5859375
test_loss: 0.24482347071170807


In [100]:
#Bilding and training GRU model
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.3,
                     recurrent_dropout=0.2,
                     return_sequences=True,
                     input_shape=(None, normalized_df1.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.3,
                     recurrent_dropout=0.2))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
    
history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=500,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/500
2/2 [==============================] - ETA: 4s - loss: 0.2520 - accuracy: 0.43 - 6s 3s/step - loss: 0.2598 - accuracy: 0.3906 - val_loss: 0.2517 - val_accuracy: 0.4773
Epoch 2/500
2/2 [==============================] - ETA: 0s - loss: 0.2563 - accuracy: 0.43 - 1s 632ms/step - loss: 0.2541 - accuracy: 0.4688 - val_loss: 0.2481 - val_accuracy: 0.5271
Epoch 3/500
2/2 [==============================] - ETA: 0s - loss: 0.2387 - accuracy: 0.65 - 1s 669ms/step - loss: 0.2441 - accuracy: 0.5938 - val_loss: 0.2504 - val_accuracy: 0.4864
Epoch 4/500
2/2 [==============================] - ETA: 0s - loss: 0.2541 - accuracy: 0.43 - 1s 600ms/step - loss: 0.2595 - accuracy: 0.4844 - val_loss: 0.2510 - val_accuracy: 0.5395
Epoch 5/500
2/2 [==============================] - ETA: 0s - loss: 0.2458 - accuracy: 0.56 - 1s 612ms/step - loss: 0.2495 - accuracy: 0.5312 - val_loss: 0.2483 - val_accuracy: 0.5501
Epoch 6/500
2/2 [==============================] - ETA: 0s - loss: 0.2620 - accuracy: 0.

Epoch 46/500
2/2 [==============================] - ETA: 0s - loss: 0.2605 - accuracy: 0.40 - 1s 711ms/step - loss: 0.2556 - accuracy: 0.4688 - val_loss: 0.2505 - val_accuracy: 0.5538
Epoch 47/500
2/2 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.46 - 1s 677ms/step - loss: 0.2512 - accuracy: 0.4531 - val_loss: 0.2519 - val_accuracy: 0.5175
Epoch 48/500
2/2 [==============================] - ETA: 0s - loss: 0.2419 - accuracy: 0.62 - 1s 605ms/step - loss: 0.2458 - accuracy: 0.5781 - val_loss: 0.2499 - val_accuracy: 0.5001
Epoch 49/500
2/2 [==============================] - ETA: 0s - loss: 0.2342 - accuracy: 0.62 - 1s 593ms/step - loss: 0.2419 - accuracy: 0.5781 - val_loss: 0.2429 - val_accuracy: 0.5003
Epoch 50/500
2/2 [==============================] - ETA: 0s - loss: 0.2606 - accuracy: 0.50 - 1s 598ms/step - loss: 0.2628 - accuracy: 0.4688 - val_loss: 0.2481 - val_accuracy: 0.4909
Epoch 51/500
2/2 [==============================] - ETA: 0s - loss: 0.2542 - acc

Epoch 91/500
2/2 [==============================] - ETA: 0s - loss: 0.2462 - accuracy: 0.65 - 1s 663ms/step - loss: 0.2487 - accuracy: 0.5469 - val_loss: 0.2440 - val_accuracy: 0.5136
Epoch 92/500
2/2 [==============================] - ETA: 0s - loss: 0.2459 - accuracy: 0.59 - 1s 702ms/step - loss: 0.2504 - accuracy: 0.5781 - val_loss: 0.2495 - val_accuracy: 0.4911
Epoch 93/500
2/2 [==============================] - ETA: 0s - loss: 0.2401 - accuracy: 0.59 - 1s 657ms/step - loss: 0.2415 - accuracy: 0.5781 - val_loss: 0.2530 - val_accuracy: 0.4858
Epoch 94/500
2/2 [==============================] - ETA: 0s - loss: 0.2600 - accuracy: 0.46 - 1s 732ms/step - loss: 0.2626 - accuracy: 0.4531 - val_loss: 0.2501 - val_accuracy: 0.4961
Epoch 95/500
2/2 [==============================] - ETA: 0s - loss: 0.2543 - accuracy: 0.56 - 1s 658ms/step - loss: 0.2532 - accuracy: 0.5469 - val_loss: 0.2507 - val_accuracy: 0.4953
Epoch 96/500
2/2 [==============================] - ETA: 0s - loss: 0.2549 - acc

2/2 [==============================] - ETA: 0s - loss: 0.2426 - accuracy: 0.59 - 1s 601ms/step - loss: 0.2529 - accuracy: 0.5312 - val_loss: 0.2527 - val_accuracy: 0.5127
Epoch 136/500
2/2 [==============================] - ETA: 0s - loss: 0.2445 - accuracy: 0.65 - 1s 586ms/step - loss: 0.2527 - accuracy: 0.5781 - val_loss: 0.2512 - val_accuracy: 0.5358
Epoch 137/500
2/2 [==============================] - ETA: 0s - loss: 0.2507 - accuracy: 0.59 - 1s 589ms/step - loss: 0.2509 - accuracy: 0.5781 - val_loss: 0.2503 - val_accuracy: 0.5317
Epoch 138/500
2/2 [==============================] - ETA: 0s - loss: 0.2585 - accuracy: 0.53 - 1s 590ms/step - loss: 0.2604 - accuracy: 0.4688 - val_loss: 0.2578 - val_accuracy: 0.5176
Epoch 139/500
2/2 [==============================] - ETA: 0s - loss: 0.2579 - accuracy: 0.43 - 1s 585ms/step - loss: 0.2527 - accuracy: 0.4844 - val_loss: 0.2512 - val_accuracy: 0.5229
Epoch 140/500
2/2 [==============================] - ETA: 0s - loss: 0.2457 - accuracy: 0

2/2 [==============================] - ETA: 0s - loss: 0.2438 - accuracy: 0.62 - 1s 589ms/step - loss: 0.2523 - accuracy: 0.5156 - val_loss: 0.2503 - val_accuracy: 0.4999
Epoch 180/500
2/2 [==============================] - ETA: 0s - loss: 0.2429 - accuracy: 0.59 - 1s 673ms/step - loss: 0.2481 - accuracy: 0.5469 - val_loss: 0.2568 - val_accuracy: 0.4956
Epoch 181/500
2/2 [==============================] - ETA: 0s - loss: 0.2456 - accuracy: 0.59 - 1s 595ms/step - loss: 0.2444 - accuracy: 0.5938 - val_loss: 0.2513 - val_accuracy: 0.4956
Epoch 182/500
2/2 [==============================] - ETA: 0s - loss: 0.2492 - accuracy: 0.53 - 1s 591ms/step - loss: 0.2535 - accuracy: 0.4688 - val_loss: 0.2394 - val_accuracy: 0.4915
Epoch 183/500
2/2 [==============================] - ETA: 0s - loss: 0.2525 - accuracy: 0.37 - 1s 593ms/step - loss: 0.2522 - accuracy: 0.4688 - val_loss: 0.2486 - val_accuracy: 0.4909
Epoch 184/500
2/2 [==============================] - ETA: 0s - loss: 0.2575 - accuracy: 0

2/2 [==============================] - ETA: 0s - loss: 0.2466 - accuracy: 0.62 - 1s 578ms/step - loss: 0.2531 - accuracy: 0.5156 - val_loss: 0.2507 - val_accuracy: 0.5183
Epoch 224/500
2/2 [==============================] - ETA: 0s - loss: 0.2436 - accuracy: 0.59 - 1s 586ms/step - loss: 0.2466 - accuracy: 0.5625 - val_loss: 0.2392 - val_accuracy: 0.4915
Epoch 225/500
2/2 [==============================] - ETA: 0s - loss: 0.2409 - accuracy: 0.56 - 1s 604ms/step - loss: 0.2435 - accuracy: 0.5781 - val_loss: 0.2488 - val_accuracy: 0.4909
Epoch 226/500
2/2 [==============================] - ETA: 0s - loss: 0.2509 - accuracy: 0.53 - 1s 595ms/step - loss: 0.2510 - accuracy: 0.5000 - val_loss: 0.2555 - val_accuracy: 0.4904
Epoch 227/500
2/2 [==============================] - ETA: 0s - loss: 0.2527 - accuracy: 0.50 - 1s 594ms/step - loss: 0.2477 - accuracy: 0.5625 - val_loss: 0.2489 - val_accuracy: 0.4917
Epoch 228/500
2/2 [==============================] - ETA: 0s - loss: 0.2425 - accuracy: 0

2/2 [==============================] - ETA: 0s - loss: 0.2411 - accuracy: 0.62 - 1s 598ms/step - loss: 0.2478 - accuracy: 0.5312 - val_loss: 0.2468 - val_accuracy: 0.5135
Epoch 268/500
2/2 [==============================] - ETA: 0s - loss: 0.2439 - accuracy: 0.59 - 1s 598ms/step - loss: 0.2492 - accuracy: 0.5625 - val_loss: 0.2552 - val_accuracy: 0.5080
Epoch 269/500
2/2 [==============================] - ETA: 0s - loss: 0.2432 - accuracy: 0.59 - 1s 604ms/step - loss: 0.2459 - accuracy: 0.5938 - val_loss: 0.2497 - val_accuracy: 0.5094
Epoch 270/500
2/2 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.53 - 1s 598ms/step - loss: 0.2535 - accuracy: 0.4688 - val_loss: 0.2506 - val_accuracy: 0.5089
Epoch 271/500
2/2 [==============================] - ETA: 0s - loss: 0.2570 - accuracy: 0.43 - 1s 597ms/step - loss: 0.2526 - accuracy: 0.4844 - val_loss: 0.2594 - val_accuracy: 0.5088
Epoch 272/500
2/2 [==============================] - ETA: 0s - loss: 0.2491 - accuracy: 0

Epoch 00310: early stopping


In [101]:
#evaluating the model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=4)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5546875
test_loss: 0.2419198900461197
